# TP5 : Alignement 2 à 2

### l. Matrices de score

In [1]:
import numpy as np

nameFile = "BLOSUM62.txt"

In [2]:
def readBlosum(nameFile):

    Blosum={}
    M = []

    flux = open(nameFile)
    for line in flux:
        line = line.rstrip().lstrip()
        if line[0] != '#':
            line = line.split(None, 24)
            M.append(line)

    M = np.array(M)

    for i in range(1, len(M)):
        lettre1 = str(M[i][0])
        for j in range(len(M[1])-1):
            lettre2 = str(M[0][j])
            Blosum[lettre1 + lettre2] = M[i][j+1]
    return Blosum

Blosum = (readBlosum(nameFile))
print('Le score de la paire ** est :', Blosum["**"])

Le score de la paire ** est : 1


In [3]:
def sousblosum(Blosum, L_aa1, L_aa2):
    
    blosum ={}
    
    for aa1 in L_aa1:
        for aa2 in L_aa2:
            for key in Blosum.keys():
                if aa1 + aa2 == key:
                    blosum[key] = Blosum[key]

    return blosum

L_aa1 = ['D', 'E', 'K']
L_aa2 = L_aa1
blosum1 = sousblosum(Blosum, L_aa1, L_aa2)

L_aa1 = ['V', 'I', 'L']
L_aa2 = L_aa1
blosum2 = sousblosum(Blosum, L_aa1, L_aa2)

L_aa1 = ['D', 'E', 'K']
L_aa2 = ['V', 'I', 'L']
blosum3 = sousblosum(Blosum, L_aa1, L_aa2)

print(blosum1, '\n\n', blosum2, '\n\n', blosum3)

{'DD': '6', 'DE': '2', 'DK': '-1', 'ED': '2', 'EE': '5', 'EK': '1', 'KD': '-1', 'KE': '1', 'KK': '5'} 

 {'VV': '4', 'VI': '3', 'VL': '1', 'IV': '3', 'II': '4', 'IL': '2', 'LV': '1', 'LI': '2', 'LL': '4'} 

 {'DV': '-3', 'DI': '-3', 'DL': '-4', 'EV': '-2', 'EI': '-3', 'EL': '-3', 'KV': '-2', 'KI': '-3', 'KL': '-2'}


In [4]:
def moyenne(Blosum):
    S = 0
    for s in Blosum.values():
        S+=int(s)
    return S/len(Blosum)

moy = moyenne(Blosum)
print('Le score moyen de la matrice BLOSUM62 est :', moy, '\n')

moy1 = moyenne(blosum1)
print('Le score moyen de la sous matrice chargée est :', moy1, '\n')

moy2 = moyenne(blosum2)
print('Le score moyen de la sous matrice hydrophobe est :', moy2, '\n')

moy3 = moyenne(blosum3)
print('Le score moyen de la sous matrice chargée + hydrophobes est :', moy3)

Le score moyen de la matrice BLOSUM62 est : -1.2604166666666667 

Le score moyen de la sous matrice chargée est : 2.2222222222222223 

Le score moyen de la sous matrice hydrophobe est : 2.6666666666666665 

Le score moyen de la sous matrice chargée + hydrophobes est : -2.7777777777777777


Ces résultats nous permettent de conclure que seuls, les acides aminés chargés et hydrophobes ont un score plutôt positifs entre eux mais quand on les oppose leur score devient négatif. Il est plus négatif que celui de leur association et c'est ce qui entrâine une score moyen négatif pour la matrice BLOSUM62 au complet.  
On en conclue donce que la matrice BLOSUM62 base les scores d'associations sur des propriétés physico-chimiques des acides aminés telle que leur hydrophobicité.

### ll. Alignement global de deux séquences par Programmation dynamique

In [5]:
seq1 = 'TG'
seq2 = 'TGA'

In [6]:
def constMat(seq1, seq2):
    
    n = len(seq1)
    m = len(seq2)
    mat = []
    
    mat.append([-k for k in range(1, m+1)])
    mat[0].insert(0, 0)
    
    for i in range(n):
        L = [-i-1]
        for j in range(1, m+1):
            L.append(0)
        mat.append(L)
    return np.array(mat)

mat = constMat(seq1, seq2)
print(mat)

[[ 0 -1 -2 -3]
 [-1  0  0  0]
 [-2  0  0  0]]


In [7]:
def bestScore(mat, i, j, seq1, seq2, gap):
    nt1 = seq1[i-1]
    nt2 = seq2[j-1]
    score = max(mat[i-1, j-1] + int(Blosum[nt1+nt2]), mat[i-1, j] + gap, mat[i, j-1] + gap)
        
    if score == mat[i-1, j-1] + int(Blosum[nt1+nt2]):
        cell = 'diag'
        
    elif score == mat[i-1, j] + gap:
        cell = 'up'
        
    else :
        cell = 'next'
        
    return (score, cell)

print(bestScore(mat, 1, 3, seq1, seq2, -1))

(-1, 'next')


In [8]:
def aliNWS(mat, seq1, seq2, gap):
    n = len(mat)
    m = len(mat[0])
    cases = []
    
    for i in range(1, n):
        L = []
        for j in range(1, m):
            score, cell = bestScore(mat, i, j, seq1, seq2, gap)
            mat[i, j] = score
            L.append(cell)
        cases.append(L)
            
    return mat, np.array(cases)

mat, cases = aliNWS(mat, seq1, seq2, -1)
print(mat,'\n\n', cases)

[[ 0 -1 -2 -3]
 [-1  5  4  3]
 [-2  4 11 10]] 

 [['diag' 'next' 'next']
 ['up' 'diag' 'next']]


In [17]:
def cheminAli(cases, seq1, seq2):
    
    ali1 = ''
    ali2 = ''
    i, j = ( len(seq1), len(seq2) )
    
    while i > 0 and j > 0:
        if cases[i-1, j-1] == 'diag' :
            i -=1
            j -=1
            ali1 += seq1[i]
            ali2 += seq2[j]
            
        elif cases[i-1, j-1] == 'up' :
            i -=1
            ali1 += seq1[i]
            ali2 += '_'
            
        else :
            j -=1
            ali1 += '_'
            ali2 += seq2[j]
    
    R_ali1 = [k for k in reversed(ali1)]
    R_ali2 = [k for k in reversed(ali2)]
    
    ali1 = ''
    ali2 = ''
    
    for k in range(len(R_ali1)):
        ali1 += R_ali1[k]
        ali2 += R_ali2[k]
    
    return (ali1, ali2)

print(cheminAli(cases, seq1, seq2))

('TG_', 'TGA')


In [21]:
sequence1 = ''
flux = open('S288C_YGR258C_RAD2_protein.fsa')
for line in flux:
    if line[0]!= '>':
        line = line.rstrip()
        sequence1 += line
        
sequence2 = ''
flux = open('S288C_YKL113C_RAD27_protein.fsa')
for line in flux:
    line = line.rstrip()
    if line[0]!='>':
        sequence2 += line
        
print( len(sequence1), '\n\n', len(sequence2) )

1032 

 383


In [30]:
def Needleman(sequence1, sequence2, gap):
    mat = constMat(sequence1, sequence2)
    mat, cases = aliNWS(mat, sequence1, sequence2, gap)
    ali = cheminAli(cases, sequence1, sequence2)
    return ali

ali1, ali2 = Needleman(sequence1, sequence2, -1)
print(ali1, '\n\n', ali2, '\n\n')
print('Longueur de l\' alignement :', len(ali1) )

MGV___H___SFWD_IAGPTARPVRLESLED_K____R_MAVDASI_WIYQFL_KAVRDQE_G___N_A_VK_NSHITG_FFRRICKLL_YFGIRPVFVFDGGVP_VLK_RE_TIRQRKERRQGKRESAKSTARKLLALQLQNGSNDNVKNSTPSSGSSVQIFKPQDEWDLPDIPGFKYDKEDAR_VNSNKTFEKLMNSINGDGLEDIDLDTINPASAEFEELPKATQYLILSSLRLKSRLRMGYSKEQLETIFPNSMDFSRFQIDMVKRRNFFTQKLINTTGFQDGGASKLNEEVINRISGQ_KSKEYKLTKTNNGWILGLGANDGSDAQKAIVIDDKDAGALVKQLDSNAED_GDVL__R__WDDLEDNSLKIVRHESSNATTAPQKRSNRSEDEGCDSDECEWEEVELKPKNV_KFVEDFSLKAARLPYMGQSLNNAGSKSFLDKRHDQASPSKTTPTMRISRISVEDDDEDYLKQIEEIEMMEAVQLSKMEKKPEAD__DKSKIAKPVTSKGTEARPPIVQYGLLGAQPDSKQPYHVTNLNSKSESVIKRT__SKTVLSEFRPPSQQEDKGAILTEGEQNLNFISHKIPQFDFNNENSLLFQKNTESNVSQEATK_EKSPIPEMPSWFSSTASQQLYNPYNTTNFVEDKNVRNEQESGAETTNKGSSYELLTGLNATEILERESEKESSNDENKDDDLEVLSEELFEDVPTKSQISKEAEDNDSRKVESINKEHRKPLIFDYDFSEDEEDNIVENMIKEQEEFDTFKNTTLSTSAERNVAENAFVEDELFEQQMK_DKRDSDEVTMDMIKEVQELLSRFGIPYITAPMEAEAQCAELLQLNLVDGIITDDSDVFLFGGTKIYKNMFHEKNYVEFYDAESILKLLGLDRKNMIELAQLLGSDYTNGLKGMGPVSSIEVIAEFGNLKNFKDWYNNGQFDKRKQETENKFEKDLRKKLVNNEIILDDDFPSVMVYDAYMRPEVDHDTTPFVWGVPDLDM